In [2]:
import pulp
import numpy as np

# Example data for inputs and outputs
n_dmus = 5  # Number of DMUs
n_inputs = 3  # Number of inputs
n_outputs = 2  # Number of outputs

# Input and output data for DMUs
inputs = np.array([
    [3, 2, 5],
    [6, 3, 4],
    [2, 7, 1],
    [4, 5, 2],
    [1, 4, 3]
])

outputs = np.array([
    [8, 9],
    [7, 5],
    [6, 6],
    [9, 8],
    [7, 7]
])
# Global target levelsfor inputs and outputs
global_input_targets = np.array([10, 15])  # GX_i
global_output_targets = np.array([20])    # GY_r

# Budget limit (B)
budget_limit = 20

# Preferences for weights
P_n_inputs = np.array([-1.2, -1.5])  # P^n_i for each input
P_p_inputs = np.array([1.3, 1.6])  # P^p_i for each input
P_n_outputs = np.array([1.4])      # P^n_r for each output
P_p_outputs = np.array([-1.5, -1.3])      # P^p_r for each output
P_g_inputs = np.array([1.1, 1.3, 1.2])  # P^g_i for global targets of inputs
P_g_outputs = np.array([1.2, 1.4])      # P^g_r for global targets of outputs

# Subsets of controllable and fixed inputs and outputs
Ic = [0, 1]  # Controllable inputs
If = [2]     # Fixed inputs
Oc = [0]     # Controllable outputs
Of = [1]     # Fixed outputs

# Budget-related subsets
IB = [0, 1]  # Budget-relevant inputs
OB = [0]     # Budget-relevant outputs

prob = pulp.LpProblem("Goal_Programming_DEA_Model", pulp.LpMinimize)

# Decision variables for deviations
n_i = [pulp.LpVariable(f"n_i_{i}_{j}", lowBound=0) for i in range(n_inputs) for j in range(n_dmus)]  # Negative deviations for inputs
p_i = [pulp.LpVariable(f"p_i_{i}_{j}", lowBound=0) for i in range(n_inputs) for j in range(n_dmus)]  # Positive deviations for inputs
n_r = [pulp.LpVariable(f"n_r_{r}_{j}", lowBound=0) for r in range(n_outputs) for j in range(n_dmus)]  # Negative deviations for outputs
p_r = [pulp.LpVariable(f"p_r_{r}_{j}", lowBound=0) for r in range(n_outputs) for j in range(n_dmus)]  # Positive deviations for outputs

# Slack variables for global target deviations
d_i_plus = [pulp.LpVariable(f"d_i_plus_{i}", lowBound=0) for i in range(n_inputs)]
d_r_minus = [pulp.LpVariable(f"d_r_minus_{r}", lowBound=0) for r in range(n_outputs)]

# Decision variables for peer weights
delta = [[pulp.LpVariable(f"delta_{j}_{k}", lowBound=0) for j in range(n_dmus)] for k in range(n_dmus)]

# Global target variables
VXi = [pulp.LpVariable(f"VX_{i}", lowBound=0) for i in range(n_inputs)]
VYr = [pulp.LpVariable(f"VY_{r}", lowBound=0) for r in range(n_outputs)]
# Objective function
prob += (
    # Minimize deviations for controllable inputs
    pulp.lpSum(P_n_inputs[i] * n_i[i * n_dmus + j] / inputs[j, i] + P_p_inputs[i] * p_i[i * n_dmus + j] / inputs[j, i]
               for i in Ic for j in range(n_dmus)) +
    # Minimize deviations for controllable outputs
    pulp.lpSum(P_n_outputs[r] * n_r[r * n_dmus + j] / outputs[j, r] + P_p_outputs[r] * p_r[r * n_dmus + j] / outputs[j, r]
               for r in Oc for j in range(n_dmus)) +
    # Minimize slack for global target inputs
    pulp.lpSum(P_g_inputs[i] * d_i_plus[i] / global_input_targets[i] for i in range(n_inputs)) +
    # Minimize slack for global target outputs
    pulp.lpSum(P_g_outputs[r] * d_r_minus[r] / global_output_targets[r] for r in range(n_outputs))
), "Objective_Function"

# Constraints: Representation of individual DMUs
for k in range(n_dmus):
    for r in Oc:
        prob += (
            pulp.lpSum(delta[k][j] * outputs[j, r] for j in range(n_dmus)) - p_r[r * n_dmus + k] + n_r[r * n_dmus + k] == outputs[k, r]
        ), f"Controllable_Output_Constraint_r_{r}_DMU_{k}"
    for i in Ic:
        prob += (
            -pulp.lpSum(delta[k][j] * inputs[j, i] for j in range(n_dmus)) + p_i[i * n_dmus + k] - n_i[i * n_dmus + k] == -inputs[k, i]
        ), f"Controllable_Input_Constraint_i_{i}_DMU_{k}"
    for r in Of:
        prob += (
            pulp.lpSum(delta[k][j] * outputs[j, r] for j in range(n_dmus)) >= outputs[k, r]
        ), f"Fixed_Output_Constraint_r_{r}_DMU_{k}"
    for i in If:
        prob += (
            -pulp.lpSum(delta[k][j] * inputs[j, i] for j in range(n_dmus)) >= -inputs[k, i]
        ), f"Fixed_Input_Constraint_i_{i}_DMU_{k}"

# Constraints: Effectiveness and global targets' achievement
for i in Ic:
    prob += (
        -pulp.lpSum(delta[j][k] * inputs[j, i] for j in range(n_dmus)) - d_i_plus[i] == -global_input_targets[i]
    ), f"Global_Target_Input_{i}"
for r in Oc:
    prob += (
        pulp.lpSum(delta[j][k] * outputs[j, r] for j in range(n_dmus)) + d_r_minus[r] == global_output_targets[r]
    ), f"Global_Target_Output_{r}"

# Constraints: Budget balance
prob += (
    pulp.lpSum(delta[j][k] * inputs[j, i] for j in range(n_dmus) for i in IB) -
    pulp.lpSum(delta[j][k] * outputs[j, r] for j in range(n_dmus) for r in OB) <= budget_limit
), "Budget_Balance"

# Solve the LP problem
prob.solve()

# Display results
print("Status:", pulp.LpStatus[prob.status])
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")

print("Objective Value:", pulp.value(prob.objective))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/83d4cecec5754b5db577cc0c168e3f3e-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/83d4cecec5754b5db577cc0c168e3f3e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 34 COLUMNS
At line 248 RHS
At line 278 BOUNDS
At line 279 ENDATA
Problem MODEL has 29 rows, 60 columns and 177 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve thinks problem is unbounded
Analysis indicates model infeasible or unbounded
0  Obj 0 Primal inf 19.69841 (23) Dual inf 40.469641 (15)
0  Obj 0 Primal inf 19.69841 (23) Dual inf 9.7199546e+11 (43)
31  Obj 1.4213131 Dual inf 1.698531 (7)
36  Obj 0.78197691 Dual inf 0.73352393 (6)
36  Obj 0.78197691 Dual inf 0.7

In [17]:
import pulp
import numpy as np

# Example setup for inputs and outputs
n_dmus = 5  # Number of DMUs
n_inputs = 5  # Number of inputs (4 controllable, 1 fixed)
n_outputs = 3  # Number of outputs (2 controllable, 1 fixed)

# Input and output data for DMUs
inputs = np.array([
    [3, 2, 5, 7, 6],
    [6, 3, 4, 9, 8],
    [2, 7, 1, 8, 7],
    [4, 5, 2, 6, 5],
    [1, 4, 3, 7, 9]
])

outputs = np.array([
    [8, 9, 5],
    [7, 5, 6],
    [6, 6, 7],
    [9, 8, 4],
    [7, 7, 8]
])

# Known global targets for controllable inputs and outputs
global_input_targets = {0: 19, 1: 25}  # GX_i: Known targets for 2 controllable inputs (indices 0, 1)
global_output_targets = {0: 40}        # GY_r: Known target for 1 controllable output (index 0)

# Budget limit
budget_limit = 100

# Preferences for controllable inputs and outputs
P_n_inputs = {0: -1.2, 1: -1.5}  # P^n_i: Preferences for negative deviations of controllable inputs
P_p_inputs = {0: 1.3, 1: 1.6}  # P^p_i: Preferences for positive deviations of controllable inputs
P_n_outputs = {0: 1.4}         # P^n_r: Preferences for negative deviations of controllable outputs
P_p_outputs = {0: -1.5}         # P^p_r: Preferences for positive deviations of controllable outputs

# Subsets of controllable and fixed inputs and outputs
Ic = [0, 1, 2, 3]  # Controllable inputs
If = [4]           # Fixed inputs
Oc = [0, 1]        # Controllable outputs
Of = [2]           # Fixed outputs

# Budget-related subsets
IB = [0, 1]  # Budget-relevant inputs
OB = [0]     # Budget-relevant outputs

# Create the LP problem
prob = pulp.LpProblem("Goal_Programming_DEA_Model", pulp.LpMinimize)

# Decision variables for deviations (specific to DMU and input/output)
n_i = {(k, i): pulp.LpVariable(f"n_i_{k}_{i}", lowBound=0) for k in range(n_dmus) for i in Ic}
p_i = {(k, i): pulp.LpVariable(f"p_i_{k}_{i}", lowBound=0) for k in range(n_dmus) for i in Ic}
n_r = {(k, r): pulp.LpVariable(f"n_r_{k}_{r}", lowBound=0) for k in range(n_dmus) for r in Oc}
p_r = {(k, r): pulp.LpVariable(f"p_r_{k}_{r}", lowBound=0) for k in range(n_dmus) for r in Oc}

# Slack variables for global targets
d_i_plus = [pulp.LpVariable(f"d_i_plus_{i}", lowBound=0) for i in global_input_targets.keys()]
d_r_minus = [pulp.LpVariable(f"d_r_minus_{r}", lowBound=0) for r in global_output_targets.keys()]

# Preferences for global targets
P_g_inputs = {0: 1.1, 1: 1.2}  # P^g_i: Preferences for global target slack variables of inputs
P_g_outputs = {0: 1.3}         # P^g_r: Preferences for global target slack variables of outputs

# Decision variables for peer weights
delta = [[pulp.LpVariable(f"delta_{k}_{j}", lowBound=0) for j in range(n_dmus)] for k in range(n_dmus)]

# Global target variables
VXi = {i: pulp.LpVariable(f"VX_{i}", lowBound=0) for i in Ic if i not in global_input_targets}
VYr = {r: pulp.LpVariable(f"VY_{r}", lowBound=0) for r in Oc if r not in global_output_targets}

# Objective function: Minimize deviations and slacks
prob += (
    # Minimize deviations for controllable inputs
    pulp.lpSum((P_n_inputs[i] * n_i[k, i]) / inputs[k, i] + (P_p_inputs[i] * p_i[k, i]) / inputs[k, i]
               for k in range(n_dmus) for i in P_n_inputs.keys()) +
    # Minimize deviations for controllable outputs
    pulp.lpSum(P_n_outputs[r] * n_r[k, r] / outputs[k, r] + P_p_outputs[r] * p_r[k, r] / outputs[k, r]
               for k in range(n_dmus) for r in P_n_outputs.keys()) +
    # Minimize slack for global target inputs
    pulp.lpSum((P_g_inputs[i]*d_i_plus[i]) / global_input_targets[i] for i in global_input_targets.keys() ) +
    # Minimize slack for global target outputs
    pulp.lpSum((P_g_outputs[r]*d_r_minus[r]) / global_output_targets[r] for r in global_output_targets.keys() )
), "Objective_Function"

# Representation of individual DMUs
for k in range(n_dmus):
    # Controllable outputs
    for r in Oc:
            prob += (
                pulp.lpSum(delta[k][j] * outputs[j, r] for j in range(n_dmus)) - p_r[k, r] + n_r[k, r] == outputs[k, r]
            ), f"Controllable_Output_{r}_DMU_{k}"
    for r in Of:
            prob += (
                pulp.lpSum(delta[k][j] * outputs[j, r] for j in range(n_dmus)) >= outputs[k, r]
            ), f"Controllable_Output_{r}_DMU_{k}"


    # Controllable inputs
    for i in Ic:
            prob += (
                -pulp.lpSum(delta[k][j] * inputs[j, i] for j in range(n_dmus)) + p_i[k, i] - n_i[k, i] == -inputs[k, i]
            ), f"Controllable_Input_{i}_DMU_{k}"
    for i in If:
            prob += (
                -pulp.lpSum(delta[k][j] * inputs[j, i] for j in range(n_dmus)) >= -inputs[k, i]
            ), f"Controllable_Input_{i}_DMU_{k}"
# Effectiveness and global targets' achievement constraints
for i in Ic:
    if i in global_input_targets:
        prob += (
            -pulp.lpSum((delta[k][j] * inputs[j, i] for k in range(n_dmus)) for j in range(n_dmus)) + d_i_plus[i] == -global_input_targets[i]
        ), f"Global_Target_Input_{i}"
    elif i in VXi:
        prob += (
            -pulp.lpSum((delta[k][j] * inputs[j, i] for k in range(n_dmus)) for j in range(n_dmus)) + VXi[i] == 0
        ), f"Estimated_Global_Target_Input_{i}"

for r in Oc:
    if r in global_output_targets:
        prob += (
            pulp.lpSum((delta[k][j] * outputs[j, r] for k in range(n_dmus)) for j in range(n_dmus)) + d_r_minus[r] == global_output_targets[r]
        ), f"Global_Target_Output_{r}"
    elif r in VYr:
        prob += (
            pulp.lpSum((delta[k][j] * outputs[j, r] for k in range(n_dmus)) for j in range(n_dmus)) - VYr[r] == 0
        ), f"Estimated_Global_Target_Output_{r}"

# Budget balance constraint
prob += (
    pulp.lpSum((delta[k][j] * inputs[j, i] for k in range(n_dmus)) for j in range(n_dmus) for i in IB) -
    pulp.lpSum((delta[k][j] * outputs[j, r] for k in range(n_dmus)) for j in range(n_dmus) for r in OB) <= budget_limit
), "Budget_Balance"

# Extra constraints for non-negativity of variables
# Deviation variables for controllable inputs
for k in range(n_dmus):
    for i in Ic:
        prob += n_i[k, i] >= 0, f"NonNegativity_n_i_{k}_{i}"
        prob += p_i[k, i] >= 0, f"NonNegativity_p_i_{k}_{i}"

# Deviation variables for controllable outputs
for k in range(n_dmus):
    for r in Oc:
        prob += n_r[k, r] >= 0, f"NonNegativity_n_r_{k}_{r}"
        prob += p_r[k, r] >= 0, f"NonNegativity_p_r_{k}_{r}"

# Delta variables (peer weights)
for k in range(n_dmus):
    for j in range(n_dmus):
        prob += delta[k][j] >= 0, f"NonNegativity_Delta_{k}_{j}"

# Global target variables for inputs (VXi) and outputs (VYr)
for i in VXi.keys():
    prob += VXi[i] >= 0, f"NonNegativity_VXi_{i}"

for r in VYr.keys():
    prob += VYr[r] >= 0, f"NonNegativity_VYr_{r}"

# Slack variables for global targets
for i in global_input_targets.keys():
    prob += d_i_plus[i] >= 0, f"NonNegativity_d_i_plus_{i}"

for r in global_output_targets.keys():
    prob += d_r_minus[r] >= 0, f"NonNegativity_d_r_minus_{r}"


# Solve the LP problem
prob.solve()

# Display results
print("Status:", pulp.LpStatus[prob.status])
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/349683612d384c4d95695e6f7d7fd9e7-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/_k/k8s6n7kj1vlchmtvlv_2jj_00000gn/T/349683612d384c4d95695e6f7d7fd9e7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 143 COLUMNS
At line 709 RHS
At line 848 BOUNDS
At line 849 ENDATA
Problem MODEL has 138 rows, 91 columns and 527 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve thinks problem is unbounded
Analysis indicates model infeasible or unbounded
0  Obj 0 Primal inf 90.515416 (38) Dual inf 9.4941937 (15)
0  Obj 0 Primal inf 90.515416 (38) Dual inf 1.8953343e+12 (56)
37  Obj 4.6288478 Primal inf 0.25483889 (2) Dual inf 1.4456595e+10 (16)
65  Obj 4.8571429 Dual inf 0.11924982 